# Data Processing for eICU-demo dataset

Downloaded (v2.0.1) from: on 12 Feb 2026

using the sqlite3 version for this processing

## Overall Notes/Comments:

- Any information related to hospital discharge has not been exported
- Filtered by within 12 hours of admission, using an offset of 720 (the offset is counted in mins)
- Might need to be careful with the admission and other drugs, as these could leak info to the model, by giving very clear indication of the diagnosis

## Tables/fields used

Tables used in this processing:

- `patient` - for the basic patient info


In [21]:
import sqlite3
import pandas as pd
import json

MAX_OFFSET = 60 * 12 # 12 hours
db_path = "input/eicu_v2_0_1.sqlite3"
conn = sqlite3.connect(db_path)


## Obtain the basic patient data

In [22]:
query = """
SELECT
    p.patientunitstayid,
    p.age,
    p.gender,
    p.ethnicity,
    p.admissionheight,
    p.hospitaladmitsource,
    p.hospitaladmitoffset
FROM patient p
"""

df_final = pd.read_sql_query(query, conn)
df_final = df_final.set_index('patientunitstayid')

df_final['hospitaladmitsource'] = df_final['hospitaladmitsource'].replace("", "unknown")
df_final

#print(f"Duplicates in df_final: {df_final.duplicated('patientunitstayid').any()}")

,age,gender,ethnicity,admissionheight,hospitaladmitsource,hospitaladmitoffset
patientunitstayid,,,,,,
141764,87,Female,Caucasian,157.5,unknown,-2258
141765,87,Female,Caucasian,157.5,unknown,-8
143870,76,Male,Caucasian,167,Operating Room,-1
144815,34,Female,Caucasian,172.7,Emergency Department,-23
145427,61,Male,Caucasian,177.8,Emergency Department,-10
...,...,...,...,...,...,...
3351763,62,Female,Caucasian,165.1,Operating Room,-68242
3352230,41,Male,African American,177.8,Direct Admit,-1512
3352231,41,Male,African American,177.8,Direct Admit,-136


## Add the patient diagnosis

Only return the primary diagnosis

In [23]:


query = "SELECT patientunitstayid, diagnosisstring, diagnosisoffset FROM diagnosis WHERE diagnosispriority='Primary';"
df_diagnosis = pd.read_sql_query(query, conn)

# Process diagnosis string to extract the main area of diagnosis
df_diagnosis['diagnosis'] = df_diagnosis['diagnosisstring'].str.split('|', expand=True)[0]
df_diagnosis.drop(['diagnosisstring'], axis=1, inplace=True)

# only get the first primary diagnosis
df_diagnosis = df_diagnosis.sort_values(by=['patientunitstayid', 'diagnosisoffset'])
df_diagnosis = df_diagnosis.drop_duplicates(subset='patientunitstayid', keep='first')

# replace any rare diagnosis with "other"
diagnosis_counts = df_diagnosis['diagnosis'].value_counts()
df_diagnosis['diagnosis'] = df_diagnosis['diagnosis'].apply(lambda x: x if diagnosis_counts[x] >= 50 else "other")

df_diagnosis = df_diagnosis.set_index('patientunitstayid')
df_diagnosis.drop(['diagnosisoffset'], axis=1, inplace=True)
df_diagnosis


,diagnosis
patientunitstayid,
143870,cardiovascular
145427,infectious diseases
151179,cardiovascular
151867,gastrointestinal
151900,pulmonary
...,...
3351763,other
3352230,cardiovascular
3352231,cardiovascular


In [24]:
# add diagnosis to df_final use inner so any without a diagnosis will get dropped
df_final = df_final.merge(df_diagnosis, on='patientunitstayid', how='inner')

df_final

,age,gender,ethnicity,admissionheight,hospitaladmitsource,hospitaladmitoffset,diagnosis
patientunitstayid,,,,,,,
143870,76,Male,Caucasian,167,Operating Room,-1,cardiovascular
145427,61,Male,Caucasian,177.8,Emergency Department,-10,infectious diseases
151179,59,Female,Caucasian,149.9,Emergency Department,-1,cardiovascular
151867,44,Male,Caucasian,172.7,Operating Room,-252,gastrointestinal
151900,66,Female,Caucasian,165.1,Emergency Department,-23,pulmonary
...,...,...,...,...,...,...,...
3351763,62,Female,Caucasian,165.1,Operating Room,-68242,other
3352230,41,Male,African American,177.8,Direct Admit,-1512,cardiovascular
3352231,41,Male,African American,177.8,Direct Admit,-136,cardiovascular


## Add Lab Results Info


In [25]:

labs_df = pd.read_sql_query("""
    SELECT patientunitstayid, labname, labresult, labmeasurenameinterface, labresultoffset
    FROM lab
    WHERE labresultoffset < ?
    ORDER BY patientunitstayid, labresultoffset
""", conn, params=(MAX_OFFSET,))


def labs_to_json(group):

    return " | ".join([f"{r['labname']}: {r['labresult']} {r['labmeasurenameinterface']}" for _, r in group.iterrows()])


lab_json_df = (
    labs_df.groupby("patientunitstayid", group_keys=False)
           .apply(labs_to_json)
           .reset_index(name="lab_results")
)

# Merge into your main df (which already exists)
df_final = df_final.merge(lab_json_df, on="patientunitstayid", how="left")

df_final

,patientunitstayid,age,gender,ethnicity,admissionheight,hospitaladmitsource,hospitaladmitoffset,diagnosis,lab_results
0,143870,76,Male,Caucasian,167,Operating Room,-1,cardiovascular,paCO2: 35 mm Hg | Total CO2: 23 mmol/L | Base ...
1,145427,61,Male,Caucasian,177.8,Emergency Department,-10,infectious diseases,total bilirubin: 0.8 mg/dL | potassium: 4.2 mm...
2,151179,59,Female,Caucasian,149.9,Emergency Department,-1,cardiovascular,bedside glucose: 65 mg/dL | glucose: 44 mg/dL ...
3,151867,44,Male,Caucasian,172.7,Operating Room,-252,gastrointestinal,AST (SGOT): 22 Units/L | sodium: 136 mmol/L | ...
4,151900,66,Female,Caucasian,165.1,Emergency Department,-23,pulmonary,PT: 10.5 sec | PT - INR: 1 | -polys: 75 % | R...
...,...,...,...,...,...,...,...,...,...
1706,3351763,62,Female,Caucasian,165.1,Operating Room,-68242,other,sodium: 136 mmol/L | chloride: 94 mmol/L | alb...
1707,3352230,41,Male,African American,177.8,Direct Admit,-1512,cardiovascular,paCO2: 55.3 mmHg | pH: 7.295 | O2 Sat (%): 10...
1708,3352231,41,Male,African American,177.8,Direct Admit,-136,cardiovascular,-monos: 11 % | MCH: 30 pg | Hgb: 16.4 g/dL | M...
1709,3352333,72,Male,Caucasian,177.8,Other Hospital,0,gastrointestinal,pH: 7.531 | FiO2: 40 % | Base Excess: 7 mmol/...


## Finally output the df to CSV


In [26]:
df_final.to_csv('./output/eicu.csv')